### **Problem**

This project aims to automate multi-label classification of disaster-related tweets using advanced natural language processing (NLP) models. The tweets, collected during hurricane events such as Harvey, Imelda, and Beryl, contain critical information ranging from property damage to requests for assistance. Manual annotation of such large-scale data is infeasible and time-consuming. Hence, there is a need for an efficient and scalable method to identify multiple categories of disaster impact from tweet content, enabling timely and informed disaster response decisions.

---

### **Objectives**

* Understand the underlying structure of multi-label text classification.
* Implement a custom PyTorch dataset and data preprocessing pipeline using geolocation and tweet metadata.
* Train a Longformer-based multi-head neural network to predict 11 disaster impact categories.
* Apply k-fold cross-validation to ensure robust model evaluation.
* Use ensemble modeling strategies (in-domain, cross-domain, two-source, and three-source) to analyze model generalization across different hurricanes.
* Generate performance metrics such as Accuracy, Precision, Recall, and F1-Score for each label and average performance.
* Save the best model per fold and use ensemble predictions to improve reliability.



1. Import Required Libraries

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import LongformerTokenizer, LongformerModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import math


2. Set Device and Reproducibility Seed

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(64)


3. Define Save Path for Model Outputs

In [ ]:
save_path = 'Training_results/Beryl_5foldcv_Longformer_Multihead/'
os.makedirs(save_path, exist_ok=True)

4. Load Tokenizer and Base Model

In [ ]:
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model_longformer = LongformerModel.from_pretrained('allenai/longformer-base-4096')

5. Define Geolocation Utility Functions

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 3959  
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_box_area(lat1, lon1, lat2, lon2):
    side1 = haversine(lat1, lon1, lat1, lon2)
    side2 = haversine(lat1, lon1, lat2, lon1)
    return side1 * side2


6. Define Default Target List

In [ ]:
def get_target_list(target_list=[]):
    if not target_list:
        target_list = [
            'homeOwnersInsurance', 'floodInsurance', 'destroyed', 'floodDamage', 'roofDamage', 
            'tsaEligible', 'tsaCheckedIn', 'rentalAssistanceEligible', 'repairAssistanceEligible', 
            'replacementAssistanceEligible', 'personalPropertyEligible'
        ]
    return target_list


7. Define Custom Dataset Class

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.float32)
        }

8. Define Longformer Multi-Head Classification Model

In [ ]:
class LongformerDeepMultiHeadClassifier(nn.Module):
    def __init__(self, num_targets=11, hidden_dim=256):
        super(LongformerDeepMultiHeadClassifier, self).__init__()
        self.longformer = model_longformer
        self.drop = nn.Dropout(0.3)
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.longformer.config.hidden_size, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(num_targets)
        ])

    def forward(self, input_ids, attention_mask):
        outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        pooled_output = sequence_output.mean(dim=1)
        x = self.drop(pooled_output)
        return torch.cat([torch.sigmoid(head(x)) for head in self.heads], dim=1)


9. Define K-Fold Cross-Validation Training Function

In [ ]:
def kfoldcv(grouped_tweets, grouped_targets, target_list=[], max_len=256, batch_size=32, num_epochs=3):
    saved_models = []
    target_list = get_target_list(target_list)

    tweet_dict = {int(name): group['text'] for name, group in grouped_tweets}
    target_dict = {int(name): group[target_list] for name, group in grouped_targets}
    intersecting_zips = list(set(target_dict.keys()) & set(tweet_dict.keys()))
    
    paired_data = {
        name: [target_dict[name].sum().apply(lambda x: 1 if x > 0 else 0), tweet_dict[name]] for name in intersecting_zips
    }

    texts = ['\n'.join(v[1].to_list()) for v in paired_data.values()]
    labels_ = [v[0] for v in paired_data.values()]
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_results = []
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(texts, labels_)):
        print(f"Training fold {fold + 1}...")
        X_train, X_test = [texts[i] for i in train_idx], [texts[i] for i in test_idx]
        y_train, y_test = [labels_[i] for i in train_idx], [labels_[i] for i in test_idx]

        train_loader = DataLoader(CustomDataset(X_train, y_train, tokenizer, max_len), batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(CustomDataset(X_test, y_test, tokenizer, max_len), batch_size=batch_size)

        model = LongformerDeepMultiHeadClassifier(len(target_list)).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-5)
        criterion = [nn.BCELoss() for _ in range(len(target_list))]
        best_f1 = 0.0

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0.0
            for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids, attention_mask)
                loss = sum(c(outputs[:, i], labels[:, i]) for i, c in enumerate(criterion))
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            model.eval()
            val_loss = 0.0
            all_preds, all_labels = [], []
            with torch.no_grad():
                for batch in tqdm(test_loader, desc=f"Epoch {epoch+1} Validation"):
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)
                    outputs = model(input_ids, attention_mask)
                    loss = sum(c(outputs[:, i], labels[:, i]) for i, c in enumerate(criterion))
                    val_loss += loss.item()
                    all_preds.append(outputs.cpu().numpy())
                    all_labels.append(labels.cpu().numpy())

            all_preds = np.vstack(all_preds)
            all_labels = np.vstack(all_labels)
            f1 = [f1_score(all_labels[:, i], all_preds[:, i] > 0.5) for i in range(len(target_list))]
            mean_f1 = np.mean(f1)

            print(f"Epoch {epoch+1}: Train Loss={train_loss/len(train_loader):.4f}, Val Loss={val_loss/len(test_loader):.4f}, Mean F1={mean_f1:.4f}")
            if mean_f1 > best_f1:
                best_f1 = mean_f1
                best_model_path = os.path.join(save_path, f"best_bert_model_fold_{fold+1}.pth")
                torch.save(model.state_dict(), best_model_path)
                print(f"New best model saved for fold {fold+1} with F1={best_f1:.4f}")

        saved_models.append(best_model_path)
    
    return fold_results, saved_models


10. Load and Preprocess Beryl Tweet and Target Data & Start K-Fold Training on Beryl Data

In [11]:
size_threshold = 80
tweets2 = pd.read_csv('organized_with_zipcode.csv')

tweets_beryl = tweets2[tweets2.storm_name == 'beryl']
bbu_areas = tweets_beryl.place_bbox.apply(lambda x: [[float(i.strip('()')) for i in x.split(', ')][i] for i in [1,0,3,2]])
tweets_beryl = tweets_beryl.loc[((tweets_beryl.geo.apply(lambda x: 'Point' in str(x))) | (bbu_areas.apply(lambda x: get_box_area(*x)) < size_threshold)), :]
tweet_grouped_beryl = tweets_beryl.groupby('zip_code')
targets_beryl = pd.read_csv('disaster_4798.csv')
target_grouped_beryl = targets_beryl.groupby('damagedZipCode')

fold_results, saved_models = kfoldcv(tweet_grouped_beryl, target_grouped_beryl, max_len = 1024, batch_size=16, num_epochs = 100)

/tmp/job.14646635/ipykernel_391663/3012017792.py:8: DtypeWarning: Columns (9,13,28,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  targets_beryl = pd.read_csv('disaster_4798.csv')


Training fold 1...


Epoch 1 Training:   0%|          | 0/4 [00:00<?, ?it/s]/tmp/job.14646635/ipykernel_391663/1724129108.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 1 Validation: 100%|██████████| 1/1 [00:00<00:00,  2.53it/s, loss=7.44]
/scratch/user/akash.pillai.0810/.my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/scratch/user/akash.pillai.0810/.my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to n

Epoch 1: Train Loss=7.5135, Val Loss=7.4401
Mean F1 Score: 0.4064, Mean Accuracy: 0.5682, Mean Precision: 0.4602, Mean Recall: 0.4606
New best model saved for fold 1 with F1=0.4064


Epoch 2 Training:   0%|          | 0/4 [00:00<?, ?it/s]/tmp/job.14646635/ipykernel_391663/1724129108.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 2 Validation: 100%|██████████| 1/1 [00:00<00:00,  2.68it/s, loss=6.84]
/scratch/user/akash.pillai.0810/.my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/scratch/user/akash.pillai.0810/.my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to n

...

Epoch 99 Training:   0%|          | 0/4 [00:00<?, ?it/s]/tmp/job.14646635/ipykernel_391663/1724129108.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 99 Validation: 100%|██████████| 1/1 [00:00<00:00,  2.73it/s, loss=3.21]
/scratch/user/akash.pillai.0810/.my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/scratch/user/akash.pillai.0810/.my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to

Epoch 99: Train Loss=1.6633, Val Loss=3.2061
Mean F1 Score: 0.8356, Mean Accuracy: 0.8920, Mean Precision: 0.8182, Mean Recall: 0.8701


Epoch 100 Training:   0%|          | 0/4 [00:00<?, ?it/s]/tmp/job.14646635/ipykernel_391663/1724129108.py:79: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'labels': torch.tensor(label, dtype=torch.float32)
Epoch 100 Validation: 100%|██████████| 1/1 [00:00<00:00,  2.72it/s, loss=3.21]

Epoch 100: Train Loss=1.6216, Val Loss=3.2102
Mean F1 Score: 0.8356, Mean Accuracy: 0.8920, Mean Precision: 0.8182, Mean Recall: 0.8701



/scratch/user/akash.pillai.0810/.my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/scratch/user/akash.pillai.0810/.my_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
